In [ ]:
import torch
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

2.7.1+cu118
11.8
True
0
NVIDIA GeForce RTX 3060 Laptop GPU


In [ ]:
echo Running with GPU...
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118/

In [5]:
import sys
import io

sys.stdin.reconfigure(encoding='utf-8')
sys.stdout.reconfigure(encoding='utf-8')
sys.stderr.reconfigure(encoding='utf-8')


AttributeError: 'OutStream' object has no attribute 'reconfigure'

In [6]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

max_seq_length = 2048  # o lo que necesites

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)

# ⛑️ Esta línea es OBLIGATORIA para evitar el error
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)


C:\Users\yordy\AppData\Local\Temp\ipykernel_29412\163890729.py:1: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


UnicodeDecodeError: 'charmap' codec can't decode byte 0x9d in position 54388: character maps to <undefined>

In [4]:
from datasets import load_dataset

dataset = load_dataset("leonidasmv/es-prehistoric-animals-tutor-dataset")
dataset = dataset["train"]


In [5]:
def format_chat(example):
    user_msg = ""
    assistant_msg = ""

    # Recorremos los mensajes en orden
    for item in example["messages"]:
        if item["role"] == "user" and not user_msg:
            user_msg = item["content"]
        elif item["role"] == "assistant" and not assistant_msg:
            assistant_msg = item["content"]

    # Si falta alguno, se omite el ejemplo
    if not user_msg or not assistant_msg:
        return {"text": ""}

    # Formato estilo Alpaca/Instruct
    prompt = f"### Instrucción:\n{user_msg}\n\n### Respuesta:\n{assistant_msg}"
    return {"text": prompt}

# Aplicar el formateo
dataset = dataset.map(format_chat)

In [5]:
print(dataset[0]['text'])
print(dataset[0]['text'])

print(dataset[0]['messages'][0]['role'])
print(dataset[0]['messages'][0]['content'])

print(dataset[0]['messages'][1]['role'])
print(dataset[0]['messages'][1]['content'])

### Instrucción:
¿Qué es un Tyrannosaurus Rex?

### Respuesta:
¡Hola chaval! El Tyrannosaurus Rex era como el matón del barrio, pero de hace 65 millones de años, ¿vale? Imagínate un autobús de dos pisos - pues así de alto era este bicho. Sus dientes eran como plátanos gigantes, pero afilados como cuchillos. ¡Qué barbaridad! Era el rey de los dinosaurios, tío.
### Instrucción:
¿Qué es un Tyrannosaurus Rex?

### Respuesta:
¡Hola chaval! El Tyrannosaurus Rex era como el matón del barrio, pero de hace 65 millones de años, ¿vale? Imagínate un autobús de dos pisos - pues así de alto era este bicho. Sus dientes eran como plátanos gigantes, pero afilados como cuchillos. ¡Qué barbaridad! Era el rey de los dinosaurios, tío.
user
¿Qué es un Tyrannosaurus Rex?
assistant
¡Hola chaval! El Tyrannosaurus Rex era como el matón del barrio, pero de hace 65 millones de años, ¿vale? Imagínate un autobús de dos pisos - pues así de alto era este bicho. Sus dientes eran como plátanos gigantes, pero afilados c

In [6]:
def tokenize(example):
    result = tokenizer(
        example["text"],
        truncation=True,
        max_length=1024,
        padding="max_length",
        return_tensors="pt",
    )
    result["labels"] = result["input_ids"].clone()
    return {
        "input_ids": result["input_ids"][0],
        "attention_mask": result["attention_mask"][0],
        "labels": result["labels"][0]
    }

# Aplicar tokenización
dataset = dataset.map(tokenize)

Map: 100%|██████████| 1010/1010 [00:00<00:00, 1184.02 examples/s]


In [7]:
import torch
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "./fine_tuned_mistral",
    num_train_epochs = 3,
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 4,
    logging_steps = 10,
    save_steps = 200,
    save_total_limit = 2,
    learning_rate = 2e-5,
    bf16 = torch.cuda.is_bf16_supported(),  # Usar bf16 si está disponible
    fp16 = not torch.cuda.is_bf16_supported(),
    optim = "paged_adamw_8bit",
    report_to = "none"
)


In [8]:
from transformers import Trainer
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = dataset,
    tokenizer = tokenizer,
)



C:\Users\yordy\AppData\Local\Temp\ipykernel_27784\2223626072.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
import xformers
print(xformers.__version__)


0.0.30


In [12]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 1,010 | Num Epochs = 3 | Total steps = 381
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 54,525,952/7,000,000,000 (0.78% trained)


AttributeError: 'NoneType' object has no attribute 'attn_bias'